# Amazon - Google

In [1]:
import sys
sys.path.append('..')
sys.path.append('/anaconda/lib/python3.6/site-packages')
import pandas as pd
import numpy as np
from modules.preprocessing import load_word_embedding_model
from modules.preprocessing import Preprocessing
from modules.preprocessing.generate_labels import gen_labels
from modules.feature_generation.gen_similarities import similarities
from sklearn.model_selection import train_test_split
from modules.preprocessing import Preprocessor

## Run on sample data to find best model using grid search

In [2]:
'''
read data
'''
# df1 = pd.read_csv("data/amazon_google/full/Amazon.csv",engine='python')
# df2 = pd.read_csv("data/amazon_google/full/GoogleProducts.csv", engine='python')
# match_df = pd.read_csv("data/amazon_google/full/Amzon_GoogleProducts_perfectMapping.csv")

df1 = pd.read_csv("data/amazon_google/sample/amazon_sample.csv",engine='python')
df2 = pd.read_csv("data/amazon_google/sample/google_sample.csv", engine='python')
match_df = pd.read_csv("data/amazon_google/sample/amazon_google_sample_match.csv")

In [3]:
'''
specify id names
'''
df1_id = 'id'
df2_id = 'id'
match_id1 = 'idAmazon' # corresponds to df1_id
match_id2 = 'idGoogleBase' # corresponds to df2_id

'''
custom data cleaning, currently this section is for google dataset only
we still need to convert currency. right now just ignoring currency effect
'''
df2["price"] = df2.price.str.replace(r"[a-zA-Z]",'').astype(float)

'''
train/test split on input dataset
'''
#random split inputs into train/test using original dataset
df1_train, df1_test = train_test_split(df1, test_size=0.33, random_state=42)


#set index dic
df1_train_index = dict(zip(df1_train[df1_id], df1_train.reset_index().index))
df1_test_index = dict(zip(df1_test[df1_id], df1_test.reset_index().index))

'''
id column manipulation
'''
# save for later use to generate labels
df1_train_id_col = df1_train[df1_id]
df1_test_id_col = df1_test[df1_id]
df2_id_col = df2[df2_id]

match_train = match_df[match_df['idAmazon'].isin(df1_train_id_col)]
match_test = match_df[match_df['idAmazon'].isin(df1_test_id_col)]

#drop id columns because we don't need to compute id similarity
df1_train = df1_train.drop(columns = [df1_id])
df1_test = df1_test.drop(columns = [df1_id])
df2 = df2.drop(columns = [df2_id])

print('\tdf1 train shape: ', df1_train.shape, '\n',
      '\tmatch train shape: ', match_train.shape, '\n'
      '\tdf1 test shape: ', df1_test.shape, '\n'
      '\tmatch test shape: ', match_test.shape)

	df1 train shape:  (76, 4) 
 	match train shape:  (22, 2) 
	df1 test shape:  (38, 4) 
	match test shape:  (8, 2)


In [4]:
'''
preprocess both dataframes
'''

glove = load_word_embedding_model('glove')
processor = Preprocessor(special_columns=['title','manufacturer'],word_embedding_model_instance=glove)
processor.fit(df1_train,df2) #fitting on training dataset for input and on whole dataset for ref

processed_train = processor.transform(df1_train, df2)
processed_test = processor.transform(df1_test, df2)

**** df1 divide columns ****
numerical_cols :  ['price']
special_field_cols :  ['title' 'manufacturer']
word_embedding_cols :  ['title' 'description' 'manufacturer']

 **** df2 divide columns ****
numerical_cols :  ['price']
special_field_cols :  ['name' 'manufacturer']
word_embedding_cols :  ['name' 'description' 'manufacturer']


In [5]:
def gen_x(processed_data):

    '''
    get numerical data
    '''

    num_matrix_1, num_matrix_2 = processed_data["numerical"][0],processed_data["numerical"][1]
    embed_matrix_1, embed_matrix_2 = processed_data["word_embedding_fields"][0],processed_data["word_embedding_fields"][1]
    spc_matrix_1, spc_matrix_2 = processed_data["special_fields"][0],processed_data["special_fields"][1]

    '''
    calculate similarities
    '''

    num_sg_data = similarities().numerical_similarity_on_matrix(num_matrix_1,num_matrix_2,method = "scaled_gaussian")
    num_mm_data = similarities().numerical_similarity_on_matrix(num_matrix_1,num_matrix_2,method = "min_max")
    embed_tfidf_data = similarities().vector_similarity_on_matrix(embed_matrix_1,embed_matrix_2)
    #embed_mean_data = similarities().vector_similarity_on_matrix(embed_matrix_1,embed_matrix_2)
    #embed_min_data = similarities().vector_similarity_on_matrix(embed_matrix_1,embed_matrix_2)
    #embed_max_data = similarities().vector_similarity_on_matrix(embed_matrix_1,embed_matrix_2)
    spc_lav_data = similarities().text_similarity_on_matrix(spc_matrix_1,spc_matrix_2, method = "lavenshtein")
    spc_jw_data = similarities().text_similarity_on_matrix(spc_matrix_1, spc_matrix_2, method="jaro_winkler")
    spc_jc_data = similarities().text_similarity_on_matrix(spc_matrix_1, spc_matrix_2, method="jaccard")
    '''
    concatenate all data
    '''
    # only concatenate non-empty similarity matrices
    non_empty = []

    for m in num_sg_data, num_mm_data, embed_tfidf_data, spc_lav_data, spc_jw_data, spc_jc_data:
        if m.size !=0:
            non_empty.append(m)

    x = np.concatenate([i for i in non_empty], axis = 1)

    print(x.shape)

    return x

In [6]:
x_train = gen_x(processed_train)
x_test = gen_x(processed_test)

'''
save features
'''
# np.save('amz_ggl_x_train',x_train)
# #del x_train

# print("***x_train saved***")

# np.save('amz_ggl_x_test',x_test)
# #del x_test

# print("***x_test saved***")

(15200, 11)
(7600, 11)


'\nsave features\n'

In [7]:
'''
train test split
'''
# generate y labels
y_train = gen_labels(df1_train_id_col, df2_id_col, match_train, match_id1, match_id2)

# simple check to see if x and y match in size
print (y_train.shape[0] == x_train.shape[0])
print(y_train.sum() == match_train.shape[0])

# generate y labels
y_test = gen_labels(df1_test_id_col, df2_id_col, match_test, match_id1, match_id2)

# simple check to see if x and y match in size
print (y_test.shape[0] == x_test.shape[0])
print(y_test.sum() == match_test.shape[0])

True
True
True
True


In [8]:
'''
modeling
'''
col_means = np.nanmean(x_train, axis=0)
inds_train = np.where(np.isnan(x_train))
inds_test = np.where(np.isnan(x_test))
x_train[inds_train]=np.take(col_means, inds_train[1])
x_test[inds_test]=np.take(col_means, inds_test[1])

from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,  precision_score, recall_score, f1_score
#import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

# #upsample
# x_maj = x_train[y_train==0]
# x_min = x_train[y_train==1]
# x_min_upsampled = resample(x_min,n_samples=x_maj.shape[0],random_state=42)
# x_train_new = np.vstack((x_maj, x_min_upsampled))
# y_train_new = np.hstack((np.zeros(x_maj.shape[0]), np.ones(x_maj.shape[0])))

n_estimators = [300]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Class weights for class imbalance issue
class_weight = [None, "balanced", "balanced_subsample"]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': class_weight}
print(random_grid)
# Use the random grid to search for best hyperparameters
rf = RandomForestClassifier()
# Random search of parameters and use all available cores
random_search = RandomizedSearchCV(estimator=rf,
                               param_distributions=random_grid,
                               n_iter=100,
                               cv=3, verbose=2, random_state=42,
                               n_jobs=-1, scoring='f1')
random_search.fit(x_train, y_train)
print(random_search.best_params_)
print("\tMean CV f1-score : %1.3f" % random_search.best_score_ )
# fit
rf_random = random_search.best_estimator_
# rf_random = RandomForestClassifier(n_estimators=300,
#                                   min_samples_split=5,
#                                   min_samples_leaf=1,
#                                   max_features='sqrt', max_depth=90,
#                                   bootstrap=True, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)
# predict
y_pred_rf = rf_random.predict(x_test)
y_pred_prob_rf = rf_random.predict_proba(x_test)[:, 1]
# roc curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_prob_rf)
# precision, recall, f1
print('RF Sample Score (Old Definition)')
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_rf))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_rf))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_rf))
print("\tAccuracy: {}".format(sum(y_pred_rf==y_test)/len(y_test)))

{'n_estimators': [300], 'max_features': ['sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'class_weight': [None, 'balanced', 'balanced_subsample']}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.9min finished


{'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 70, 'class_weight': 'balanced_subsample', 'bootstrap': False}
	Mean CV f1-score : 0.600
RF Sample Score (Old Definition)
	Precision: 0.625
	Recall: 0.625
	F1: 0.625
	Accuracy: 0.9992105263157894


In [9]:
def gen_orig_labels(df1_id, match_id1, df2_id, match_id2, match, y_pred_prob):
    match = match.dropna()
    true_label=df1_id.isin(match[match_id1]).astype(int)
    import itertools
    res =[]
    for idx, (id_1,id_2) in enumerate(itertools.product(list(df1_id), list(df2_id))):
        res.append([id_1,id_2,y_pred_prob[idx]])
    
    match = pd.DataFrame(res, columns=[match_id1,match_id2,'prob'])
    match = match[match['prob']>0.5]
    pred_match = match.loc[match.groupby(match_id1)['prob'].idxmax()]
    pred_label = df1_test_id_col.isin(pred_match[match_id1]).astype(int)
    
    return np.array(true_label), np.array(pred_label)

In [10]:
true, pred = gen_orig_labels(df1_test_id_col, match_id1, df2_id_col, match_id2, match_test, y_pred_prob_rf)

In [11]:
print('RF Sample Score -- Aligned with Neoway')

print("\tPrecision: %1.3f" % precision_score(true, pred))
print("\tRecall: %1.3f" % recall_score(true, pred))
print("\tF1: %1.3f" % f1_score(true, pred))
print("\tAccuracy: {}".format(sum(pred==true)/len(true)))

RF Sample Score -- Aligned with Neoway
	Precision: 0.625
	Recall: 0.625
	F1: 0.625
	Accuracy: 0.8421052631578947


## Run best model on full datasets

In [ ]:
'''
read data
'''
df1 = pd.read_csv("data/amazon_google/full/Amazon.csv",engine='python')
df2 = pd.read_csv("data/amazon_google/full/GoogleProducts.csv", engine='python')
match_df = pd.read_csv("data/amazon_google/full/Amzon_GoogleProducts_perfectMapping.csv")

In [ ]:
'''
specify id names
'''
df1_id = 'id'
df2_id = 'id'
match_id1 = 'idAmazon' # corresponds to df1_id
match_id2 = 'idGoogleBase' # corresponds to df2_id

'''
custom data cleaning, currently this section is for google dataset only
we still need to convert currency. right now just ignoring currency effect
'''
df2["price"] = df2.price.str.replace(r"[a-zA-Z]",'').astype(float)

'''
train/test split on input dataset
'''
#random split inputs into train/test using original dataset
df1_train, df1_test = train_test_split(df1, test_size=0.33, random_state=42)


#set index dic
df1_train_index = dict(zip(df1_train[df1_id], df1_train.reset_index().index))
df1_test_index = dict(zip(df1_test[df1_id], df1_test.reset_index().index))

'''
id column manipulation
'''
# save for later use to generate labels
df1_train_id_col = df1_train[df1_id]
df1_test_id_col = df1_test[df1_id]
df2_id_col = df2[df2_id]

match_train = match_df[match_df['idAmazon'].isin(df1_train_id_col)]
match_test = match_df[match_df['idAmazon'].isin(df1_test_id_col)]

#drop id columns because we don't need to compute id similarity
df1_train = df1_train.drop(columns = [df1_id])
df1_test = df1_test.drop(columns = [df1_id])
df2 = df2.drop(columns = [df2_id])

print('\tdf1 train shape: ', df1_train.shape, '\n',
      '\tmatch train shape: ', match_train.shape, '\n'
      '\tdf1 test shape: ', df1_test.shape, '\n'
      '\tmatch test shape: ', match_test.shape)

In [ ]:
'''
preprocess both dataframes
'''

glove = load_word_embedding_model('glove')
processor = Preprocessor(special_columns=['title','manufacturer'],word_embedding_model_instance=glove)
processor.fit(df1_train,df2) #fitting on training dataset for input and on whole dataset for ref

processed_train = processor.transform(df1_train, df2)
processed_test = processor.transform(df1_test, df2)

In [ ]:
x_train = gen_x(processed_train)
x_test = gen_x(processed_test)

'''
save features
'''
np.save('amz_ggl_x_train',x_train)
#del x_train

print("***x_train saved***")

np.save('amz_ggl_x_test',x_test)
#del x_test

print("***x_test saved***")

In [ ]:
'''
train test split
'''
# generate y labels
y_train = gen_labels(df1_train_id_col, df2_id_col, match_train, match_id1, match_id2)

# simple check to see if x and y match in size
print (y_train.shape[0] == x_train.shape[0])
print(y_train.sum() == match_train.shape[0])

# generate y labels
y_test = gen_labels(df1_test_id_col, df2_id_col, match_test, match_id1, match_id2)

# simple check to see if x and y match in size
print (y_test.shape[0] == x_test.shape[0])
print(y_test.sum() == match_test.shape[0])

In [ ]:
'''
modeling
'''
col_means = np.nanmean(x_train, axis=0)
inds_train = np.where(np.isnan(x_train))
inds_test = np.where(np.isnan(x_test))
x_train[inds_train]=np.take(col_means, inds_train[1])
x_test[inds_test]=np.take(col_means, inds_test[1])

from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,  precision_score, recall_score, f1_score
#import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

# fit
rf_random = random_search.best_estimator_
# rf_random = RandomForestClassifier(n_estimators=300,
#                                   min_samples_split=5,
#                                   min_samples_leaf=1,
#                                   max_features='sqrt', max_depth=90,
#                                   bootstrap=True, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)
# predict
y_pred_rf = rf_random.predict(x_test)
y_pred_prob_rf = rf_random.predict_proba(x_test)[:, 1]
# roc curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_prob_rf)
# precision, recall, f1
print('RF Full Dataset Score (Old Definition)')
print("\tPrecision: %1.3f" % precision_score(y_test, y_pred_rf))
print("\tRecall: %1.3f" % recall_score(y_test, y_pred_rf))
print("\tF1: %1.3f" % f1_score(y_test, y_pred_rf))
print("\tAccuracy: {}".format(sum(y_pred_rf==y_test)/len(y_test)))

# save the classifier
import pickle
with open('amz_ggl_rf.pkl', 'wb') as fid:
    pickle.dump(rf_random, fid, protocol=4)

print("***model saved***")

In [ ]:
true, pred = gen_orig_labels(df1_test_id_col, match_id1, df2_id_col, match_id2, match_test, y_pred_prob_rf)

In [ ]:
print('RF Full Dataset Score -- Aligned with Neoway')

print("\tPrecision: %1.3f" % precision_score(true, pred))
print("\tRecall: %1.3f" % recall_score(true, pred))
print("\tF1: %1.3f" % f1_score(true, pred))
print("\tAccuracy: {}".format(sum(pred==true)/len(true)))